# Here we Convert an Topic to Blog 
## Topic -> LLM will generate outline -> outline fed into LLM -> Generate Blog

In [5]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

In [6]:
load_dotenv()

llm=ChatGoogleGenerativeAI(
model="gemini-2.5-flash",
)

E0000 00:00:1760369181.360458  246892 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [7]:
class BlogState(TypedDict):
    title:str 
    outline:str 
    content:str 
    

In [8]:
def create_outline(state:BlogState)->BlogState:
    title=state['title']
    prompt=f'Create an outline of 10-20 words which we can used for creating blog for follwoing topic : {title}'
    response=llm.invoke(prompt).content
    state['outline']=response
    return state

In [9]:
def create_blog(state:BlogState)->BlogState:
    title=state['outline']
    prompt=f'Create an blog of 50-100 words which is based on  follwoing topic : {title} . add humor , thriling ,facts in blog'
    response=llm.invoke(prompt).content
    state['content']=response
    return state

In [12]:
graph=StateGraph(BlogState)


#add node 
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)

#add edges 

graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog',END)

#compile 
workflow=graph.compile() 


In [14]:
initial_state={"title":"Life"}
output_state=workflow.invoke(initial_state)
print(output_state)

{'title': 'Life', 'outline': '**Life: Meaning, joys, challenges. Growth, purpose. Embrace change, connect. Live fully.** (16 words)', 'content': 'Life, eh? One minute you\'re pondering its deep meaning, the next you\'re Googling "why did my bread burn?" (Spoiler: it\'s rarely a cosmic challenge). Joys? Plentiful! Challenges? Like that one rogue sock in the laundry – always there.\n\nBut here\'s the thrilling part: every stumble is a chance to grow. Did you know the average person makes around 35,000 decisions daily? That\'s 35,000 opportunities for purpose! Embrace change like a cat finding a new sunbeam. Connect deeply, because humans are, scientifically, social creatures. So, laugh, learn, leap! Live fully, because this wild ride isn\'t a dress rehearsal.'}


In [15]:
print(output_state['content'])

Life, eh? One minute you're pondering its deep meaning, the next you're Googling "why did my bread burn?" (Spoiler: it's rarely a cosmic challenge). Joys? Plentiful! Challenges? Like that one rogue sock in the laundry – always there.

But here's the thrilling part: every stumble is a chance to grow. Did you know the average person makes around 35,000 decisions daily? That's 35,000 opportunities for purpose! Embrace change like a cat finding a new sunbeam. Connect deeply, because humans are, scientifically, social creatures. So, laugh, learn, leap! Live fully, because this wild ride isn't a dress rehearsal.
